In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/Github/SSM")

include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
# include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using LinearAlgebra
using Plots
using Distributions
using Random
using .SSM
using StatsBase

  Activating project at `\Users\ryansenne\Documents\Github\SSM`


In [2]:
# generate data from a Gaussian HMM model
Random.seed!(1234)

A = [0.7 0.2 0.1; 0.1 0.7 0.2; 0.2 0.1 0.7]
    means = [[0.0, 0.0], [-1.0, 2.0], [3.0, 2.5]]
    covs = [
                [0.1 0.0; 0.0 0.1],  # Covariance matrix for state 1
                [0.1 0.0; 0.0 0.1],  # Covariance matrix for state 2
                [0.1 0.0; 0.0 0.1]   # Covariance matrix for state 3
            ]
emissions_models = [GaussianEmission(mean, cov) for (mean, cov) in zip(means, covs)]
simul_hmm = GaussianHMM(A, emissions_models, [0.33, 0.33, 0.34], 3, 2)
states, observations = SSM.sample(simul_hmm, 10000)


model = GaussianHMM(observations, 3)
# baumWelch!(model, observations, 100)

GaussianHMM{GaussianEmission}([0.3630736531110074 0.46347611391430854 0.17345023297468412; 0.36016403943942515 0.6012206650376235 0.038615295522951176; 0.3526976351777686 0.15216815563956368 0.49513420918266776], GaussianEmission[GaussianEmission([-0.09559366950911044, -0.05921829347226094], [0.06908377541266905 -0.019335053180299044; -0.019335053180299044 0.0898819474893171]), GaussianEmission([-1.0041102873930032, 2.003063119889526], [0.1017011095291989 0.0007908684567223624; 0.0007908684567223624 0.09409146349095061]), GaussianEmission([2.5742184683676563, 2.130326367026472], [1.0309149461568967 0.797835536093344; 0.797835536093344 0.7905471104524638])], [0.32041658989302524, 0.4853870103273905, 0.19419639977958428], 3, 2)

In [4]:
alpha = SSM.forward(model, observations)
beta = SSM.backward(model, observations)
xi = SSM.calculate_ξ(model, alpha, beta, observations)

3×3×9999 Array{Float64, 3}:
[:, :, 1] =
 -18.1153   -0.00573578  -20.788
 -49.9552  -31.5774      -54.1222
 -22.1886   -5.1639      -23.7835

[:, :, 2] =
 -157.289  -101.658   -18.0984
 -137.697   -81.7967   -1.47884e-8
 -161.008  -106.461   -20.7392

[:, :, 3] =
 -270.359  -206.053   -137.697
 -212.964  -148.391    -81.7967
 -133.74    -70.5192     0.0

;;; … 

[:, :, 9997] =
  -0.072292  -20.5265   -4.41383
 -33.8265    -54.0125  -39.6622
  -2.89121   -24.4302   -6.15482

[:, :, 9998] =
  -0.0197633  -22.5551   -5.23303
 -20.5078     -42.7748  -27.2152
  -4.26813    -27.8882   -8.40347

[:, :, 9999] =
  -0.0167598  -32.5151   -4.50453
 -22.5528     -54.7828  -28.5348
  -5.22442    -38.8075   -8.63426

In [7]:
sum(exp.(xi[:, :, 1]))

0.9999999999992296

In [16]:
using DataFrames
using CSV
CSV.write("hmm_observations.csv", DataFrame(observations, :auto), writeheader=false)

"hmm_observations.csv"

In [14]:
using CSV
using DataFrames

# import old faithful data
old_faithful = CSV.read("/Users/ryansenne/PycharmProjects/NV_States_GLM_HMM/HMM_Bishop/faithful.csv", DataFrame)

# set up a variable for the HMM
X = Matrix{Float64}(old_faithful[:, [:waiting, :eruptions]])
#  zscore the data
X = (X .- mean(X, dims=1)) ./ std(X, dims=1)

# Set up initial parameters
A = [0.2 0.8; 0.5 0.5];
Pi = [0.5, 0.5];

U1 = [3, -5];
U2 = [-3, -3];
Sigma1 = [1 0; 0 1];
Sigma2 = [0.5 0; 0 0.5];

hmm = SSM.GaussianHMM(A, [GaussianEmission(U1, Sigma1), GaussianEmission(U2, Sigma2)], Pi, 2, 2)
# hmm = SSM.GaussianHMM(X, 2)
baumWelch!(hmm, X, 11, 1e-3)


-4584.5258332169415
-386.3132837521509
-370.538737764364
-361.912908169515
-357.8273503052301
-353.49494957458353
-350.7938428520957
-350.33259560079733
-350.3002596397879
-350.2990049814304
-350.29896245266696


In [15]:
hmm.A

2×2 Matrix{Float64}:
 0.476748  0.523252
 0.938164  0.0618363

In [8]:
SSM.loglikelihood(hmm.B[2], X[2, :])

-6.546418900933308

In [11]:
SSM.backward(hmm, X)'

272×2 adjoint(::Matrix{Float64}) with eltype Float64:
 -4566.11    -4566.58
 -4560.09    -4559.34
 -4541.2     -4541.67
 -4531.47    -4530.55
 -4508.6     -4509.07
 -4498.69    -4497.77
 -4475.35    -4474.44
 -4456.98    -4457.45
 -4451.44    -4450.52
 -4429.5     -4429.97
     ⋮       
  -117.004    -117.474
  -111.44     -111.909
  -102.426    -101.569
   -76.7781    -75.8624
   -55.3766    -55.8466
   -50.2179    -49.3016
   -28.5968    -29.0668
   -24.3439    -23.5034
     0.0         0.0